# Keras Network to Classify Odd/Even Numbers

In [1]:
import warnings
warnings.filterwarnings('ignore')

# imports
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Model

np.random.seed(23)

## Big Picture
- Supervised learning, so we have labeled data
- Neural networks, like other supervised algorithms, try to approximate any arbitrary function, given enough data

<img src="machine_learning_vs_classical_programming.jpeg">

## Goal
- Train a neural network classifier to classify if an integer is odd or even.

## Workflow
1. Setup training data
2. Create binary encoding for the input layer
3. Create binary decoding for the ouput layer
4. Create the model, add layers, and train/fit
5. Create predictions on new data
6. Evaluate the performance of the classifier on train data
7. Evaluate the performance of the classifier on test data

In [2]:
# Specify the number of binary digits. 
# How many possible digits we have in the hypothesis space, the space where we have representations of those layers
# Think about how we can show 10000 on ten fingers, with binary being whether or not that number is present on that finger

NUM_DIGITS = 10

In [3]:
## Setup the training data for 101-1023. 1023 is the highest number countable with 10 binary digits

raw_training_data = np.array(range(101, 2**NUM_DIGITS))

In [4]:
#101 till 1024
raw_training_data[:10]

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])

In [5]:
raw_training_data[:10] , raw_training_data[-5:]

(array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110]),
 array([1019, 1020, 1021, 1022, 1023]))

In [6]:
## Binary Encoding function
# In order to binary encode the input values
# takes in digit in base ten and the number of binary digits we need to convert it to
def binary_encode(i, NUM_DIGITS):
    return np.array([i >> d & 1 for d in range(NUM_DIGITS)])

In [7]:
print("number \t [binary digits]")
print(1, "\t", binary_encode(1, NUM_DIGITS))
print(2, "\t", binary_encode(2, NUM_DIGITS))
print(3, "\t", binary_encode(3, NUM_DIGITS))
print(4, "\t", binary_encode(4, NUM_DIGITS))
print(4, "\t", binary_encode(5, NUM_DIGITS))

number 	 [binary digits]
1 	 [1 0 0 0 0 0 0 0 0 0]
2 	 [0 1 0 0 0 0 0 0 0 0]
3 	 [1 1 0 0 0 0 0 0 0 0]
4 	 [0 0 1 0 0 0 0 0 0 0]
4 	 [1 0 1 0 0 0 0 0 0 0]


In [8]:
print(0, "\t", binary_encode(0, NUM_DIGITS))
print(1023, "\t", binary_encode(1023, NUM_DIGITS))

0 	 [0 0 0 0 0 0 0 0 0 0]
1023 	 [1 1 1 1 1 1 1 1 1 1]


In [9]:
512 + 256 + 128 + 64 + 32 + 16 + 8 + 4 + 2 + 1

1023

In [10]:
# encode the training data
encoded_training_data = [binary_encode(i, NUM_DIGITS) for i in raw_training_data]

In [11]:
x_train = np.array(encoded_training_data)

In [12]:
x_train

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [13]:
print(x_train[0])
print(raw_training_data[0])

[1 0 1 0 0 1 1 0 0 0]
101


In [14]:
# Encode "even" and "odd" to zero or 1
# If we had more than 2 options, we'd one hot encode to make an array
# One hot encoding the number
# Creating the labels, the y values
def encode_even_odd(i):
    if i % 2 == 0:
        return [0]
    else:
        return [1]

assert encode_even_odd(2) == [0]
assert encode_even_odd(3) == [1]

In [15]:
# y_train is the encoded output. 
# This is our "labeled data" for supervised learning
y_train = np.array([encode_even_odd(i) for i in range(101, 2 ** NUM_DIGITS)])

In [16]:
print("Number representeed as a decimal", raw_training_data[0])
print("Number represented as binary in a list:", x_train[0])
print("0 for even, 1 for odd:", y_train[0])

Number representeed as a decimal 101
Number represented as binary in a list: [1 0 1 0 0 1 1 0 0 0]
0 for even, 1 for odd: [1]


In [17]:
print("Number representeed as a decimal", raw_training_data[1])
print("Number represented as binary in a list:", x_train[1])
print("0 for even, 1 for odd:", y_train[1])

Number representeed as a decimal 102
Number represented as binary in a list: [0 1 1 0 0 1 1 0 0 0]
0 for even, 1 for odd: [0]


## Compare raw inputs to the lables
- The first five values in the input tensor are 101, 102, 103, 104, and 105
- The encoded labels specify "odd", "even", "odd", "even", and then "odd"
- Remember that training a classifier is supervised learning with labeled data

In [18]:
print("Raw training data", raw_training_data[:5])
print(y_train[:5].flatten())

Raw training data [101 102 103 104 105]
[1 0 1 0 1]


In [20]:
# Convert from binary back to the ouput string
# Decodes the output layer to English
# Decoding our findings
def decode_even_odd(n):
    if n == 0:
        return "even"
    else:
        return "odd"

assert decode_even_odd(0) == 'even'
assert decode_even_odd(1) == 'odd'

In [21]:
# Another way I've seen this decoding written (especially with one hot encoded options)
# Convert from binary back to the ouput string
def decode_even_odd2(n):
    return ["even", "odd"][n]

assert decode_even_odd2(0) == 'even'
assert decode_even_odd2(1) == 'odd'

In [22]:
# Now let's build our model, add layers, compile, and fit it!
# Sequential is a good model for sequence data like 1, 2, 3 dimensional arrays
# good choice if all inputs and outputs are a single value
model = Sequential()

## Add the model's layers.
# These are defaults and they work pretty well
model.add(Dense(1000, input_dim=NUM_DIGITS, activation="relu")) # relu max(0, x)
model.add(Dense(1000, activation="relu"))

# Almost all the time, the last layer's first argument is the number of outcomes
model.add(Dense(2, activation="softmax"))

# categorical_crossentropy for multiclass
# should be using a binary classifier 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=["accuracy"])
#loss is difference between the actual and predicted

In [23]:
# Fit the model on the training data
# nb_epoch is number of training loops is number of epochs

model.fit(x_train, y_train, epochs=3, batch_size=128)

Epoch 1/3
8/8 [==============================] - 10s 6ms/step - loss: 0.6943 - accuracy: 0.4984
Epoch 2/3
8/8 [==============================] - 0s 6ms/step - loss: 0.6903 - accuracy: 0.5436
Epoch 3/3
8/8 [==============================] - 0s 6ms/step - loss: 0.6854 - accuracy: 0.6356


In [26]:
# crank it up

model.fit(x_train, y_train, epochs=20, batch_size=128)

Epoch 1/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6349 - accuracy: 0.8462
Epoch 2/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6311 - accuracy: 0.8418
Epoch 3/20
8/8 [==============================] - 0s 8ms/step - loss: 0.6273 - accuracy: 0.8700
Epoch 4/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6235 - accuracy: 0.8938
Epoch 5/20
8/8 [==============================] - 0s 6ms/step - loss: 0.6197 - accuracy: 0.9079
Epoch 6/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6159 - accuracy: 0.9231
Epoch 7/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6121 - accuracy: 0.9317
Epoch 8/20
8/8 [==============================] - 0s 6ms/step - loss: 0.6083 - accuracy: 0.9426
Epoch 9/20
8/8 [==============================] - 0s 7ms/step - loss: 0.6045 - accuracy: 0.9469
Epoch 10/20
8/8 [==============================] - 0s 6ms/step - loss: 0.6007 - accuracy: 0.9567
Epoch 11/20
8/8 [======================

In [24]:
y_train[0:3]

array([[1],
       [0],
       [1]])

In [27]:
# Setup x_test for numbers from 1 to 100 (the model hasn't seen these at all)
numbers = np.arange(1, 101)
x_test = np.transpose(binary_encode(numbers, NUM_DIGITS))

In [28]:
x_test[0:3]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [29]:
# model.predict_classes(x_test) or model.predict_classes(out_of_sample_data)
y_test = model.predict_classes(x_test)

In [30]:
y_test[0:3]

array([1, 0, 1])

In [31]:
# Setup predicted output
predictions = np.vectorize(decode_even_odd)(y_test) # np.vectorize is like .apply on a pandas series
print (predictions)

['odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even'
 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd' 'even' 'odd'
 'even']


In [32]:
# lets produce our array of actual values to compare out predicted values
actual = np.array([])
for i in numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even',
       'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd', 'even', 'odd',
       'even'], dtype='<U32')

In [33]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

1.0


### Ok, 100% accuracy... but what if we send the model random numbers?

In [34]:
random_numbers = np.random.randint(1023, size=10000)
random_numbers[:10]

array([595, 742,  40, 969, 950, 488,  31, 237, 460, 347])

In [35]:
x_test = np.transpose(binary_encode(random_numbers, NUM_DIGITS))
y_test = model.predict_classes(x_test)

In [36]:
# turn out vectors of numbers into srings for human readable output
predictions = np.vectorize(decode_even_odd)(y_test)

In [37]:
predictions.shape

(10000,)

In [38]:
actual = np.array([])
for i in random_numbers:
    if i % 2 == 0:
        actual = np.append(actual, "even")
    else:
        actual = np.append(actual, "odd")

actual

array(['odd', 'even', 'even', ..., 'even', 'even', 'odd'], dtype='<U32')

In [39]:
actual.shape

(10000,)

In [40]:
# Let's evaluate the model's predictions
evaluate = np.array(actual == predictions)
print (np.count_nonzero(evaluate == True) / 100)

100.0


In [ ]:
# make predictions on 1-100 
# make predictions on 1024-2048
# make predictions on 2048-4096

- experiment with the number of binary digits
- play with the number of epochs